# Coding Question Difficulty Prediction

This notebook is a project with the aim of predicting difficulty in answering competitive coding questions using labelled data from [codeforces.com](http://codeforces.com/).

It classifies problems into three levels of difficulty- Easy, Medium, and Hard from a dataset of around 2000 questions containing - The question text, the input, and the output specifications

## Imports

In [2]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM, Dropout
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
import keras.utils

import nltk
import json
import glob
import os.path
import pprint

C:\Users\Nikhil Prabhu\AppData\Local\Programs\Python\Python36\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
GLOVE_PATH = 'glove_50_glove_100'

## Data Processing

In [4]:
def tokenize_and_lower(string):
    return " ".join(nltk.word_tokenize(string)).lower()

In [5]:
def load_data():
    '''
    reads all question files and returns - 
    q_text, input, output, label
    '''

    complexities = dict()
    complexity_file = open("questions-complexity.csv", encoding='utf-8')
    complexity_file.readline()

    for line in complexity_file:
        line = line.strip().split(",")
        complexities[line[0]] = line[-2]
    complexity_file.close()


    question_files = sorted(glob.glob("./questions/*.txt"))
    questions = []
    inputs = []
    outputs = []
    labels = []

    for f in question_files:
        handle = open(f, encoding='utf-8')
        text = handle.read()
        handle.close()
        text_split = text.split("\n\n")
        question = text_split[2]	### + [3] and [4] for Input and Output requirement text
        input_text = text_split[3]
        output_text = text_split[4]

        # Removes 'Input' and 'Output' prefixes
        input_text = input_text[len('Input'):]
        output_text = output_text[len('Output'):]

        question = tokenize_and_lower(question)
        input_text = tokenize_and_lower(input_text)
        output_text = tokenize_and_lower(output_text)

        questions.append(question)
        inputs.append(input_text)
        outputs.append(output_text)
        labels.append(complexities[os.path.basename(f).strip(".txt")])

    return questions, inputs, outputs, labels

## Feature Extraction

In [48]:
def get_categories(labels):
    labels = [0 if l == "Easy" else 1 if l == "Medium" else 2 if l == "Hard" else l for l in labels]
    labels_vector = keras.utils.to_categorical(labels)
    return labels_vector

### Word Frequency feature extraction

In [49]:
def populate_word_frequencies(word_freq, text):
    for token in nltk.word_tokenize(text):
        if token not in word_freq:
            word_freq[token] = 0
        else:
            word_freq[token] += 1
    return word_freq

In [50]:
def create_vocab(questions, inputs, outputs):
    '''
    takes list of questions (returned from load_data) and returns a list of words sorted in desending order of frequency
    '''
    word_freq = dict()
    for ques in questions:
        populate_word_frequencies(word_freq, ques)
            
    for input_text in inputs:
        populate_word_frequencies(word_freq, input_text)
        
    for output_text in outputs:
        populate_word_frequencies(word_freq, output_text)
        
    top_words = sorted(word_freq, key = lambda w : word_freq[w], reverse = True)
    vocab_size = len(top_words)

    return top_words, vocab_size

In [51]:
def get_frequency_vector(text, vocab):
    vector = []
    for token in nltk.word_tokenize(text):
        vector.append(vocab.index(token))
    return vector

In [52]:
def get_frequency_vectors(questions, inputs, outputs, vocab):
    question_vectors = []
    input_vectors = []
    output_vectors = []
    
    for ques in questions:
        question_vectors.append(get_frequency_vector(ques, vocab))

    for input_text in inputs:
        input_vectors.append(get_frequency_vector(input_text, vocab))
        
    for output_text in outputs:
        output_vectors.append(get_frequency_vector(output_text, vocab))
        
    return question_vectors, input_vectors, output_vectors

### Word Embedding Feature Extraction

In [53]:
vecs = np.load("glove_50_glove_100/glove_vectors_100d.npy")
vecs50 = np.load("glove_50_glove_100/glove_vectors_50d.npy")

In [54]:
with open('glove_50_glove_100/words.txt', encoding='utf-8') as f:
    content = f.readlines()
words = [x.strip() for x in content] 

In [55]:
wordidx = json.load(open('glove_50_glove_100/wordsidx.txt'))

In [80]:
def get_embedding_vector(text, embedding_size, vecs, vecs50, wordidx):
    vector = []
    for token in nltk.word_tokenize(text.lower()):
        if embedding_size == '50':
            try:
                vector.append(vecs50[wordidx[token]])
            except:
                print('No vector present:', token)
        elif embedding_size == '100':
            try:
                vector.append(vecs[wordidx[token]])
            except:
                print('No vector present:', token)
    
    vector = np.concatenate(vector) if vector else np.array([0.0])

    return vector 

In [81]:
x = get_embedding_vector('Hi how are you ?', '50', vecs, vecs50, wordidx)
len(x)
y = np.array([-1.0])
-1 in y

True

In [82]:
def get_embedding_vectors(questions, inputs, outputs, vecs, vecs50, wordidx, embedding_size='50'):
    question_vectors = []
    input_vectors = []
    output_vectors = []
    
    for ques in questions:
        question_vectors.append(get_embedding_vector(ques, embedding_size, vecs, vecs50, wordidx))
        
    for input_text in inputs:
        input_vectors.append(get_embedding_vector(input_text, embedding_size, vecs, vecs50, wordidx))
        
    for output_text in outputs:
        output_vectors.append(get_embedding_vector(output_text, embedding_size, vecs, vecs50, wordidx))
        
#     print('q:', question_vectors[0])
    return question_vectors, input_vectors, output_vectors

## Feature Selection

In [83]:
algorithm = 'embeddings'
# algorithm = 'frequencies'

In [85]:
vecs[wordidx['ha']]

array([-0.21568  ,  0.21229  ,  0.17246  ,  0.52203  , -0.54868  ,
       -0.096671 ,  0.54394  ,  0.93758  ,  1.2568   ,  0.5532   ,
        0.78287  ,  0.75195  ,  0.0038273, -0.10445  ,  1.2246   ,
       -0.87819  ,  0.55927  ,  0.37283  ,  0.058181 , -0.024589 ,
        1.382    ,  1.2649   , -0.79755  ,  0.19032  ,  0.42323  ,
        0.32378  ,  0.40399  , -0.2634   , -0.23317  , -0.15091  ,
       -0.7121   ,  0.09766  ,  1.0384   ,  0.31502  , -0.2307   ,
        0.2184   ,  0.78312  ,  0.47516  , -0.53914  ,  0.29622  ,
        0.26135  , -0.051061 , -0.15684  , -0.082803 ,  1.3629   ,
       -0.22898  , -1.1811   , -0.65929  , -1.0497   , -0.56452  ,
       -1.1406   , -0.28304  ,  0.31525  ,  0.29522  , -0.69138  ,
        0.18481  ,  0.028495 ,  0.23261  ,  0.34459  , -0.59282  ,
       -0.16287  ,  0.82126  , -0.13743  , -1.0789   ,  0.68146  ,
       -0.54496  ,  0.79759  , -0.048629 ,  0.69745  ,  0.272    ,
        0.83907  ,  0.73915  ,  0.21814  , -0.80616  , -0.1149

In [84]:
questions, inputs, outputs, labels = load_data()
labels = get_categories(labels)
vocab, vocab_size = create_vocab(questions, inputs, outputs)
if algorithm == 'frequencies':
#     vocab, vocab_size = create_vocab(questions, inputs, outputs)
    question_vectors, input_vectors, output_vectors = get_frequency_vectors(questions, inputs, outputs, vocab)
elif algorithm == 'embeddings':
    question_vectors, input_vectors, output_vectors = get_embedding_vectors(questions, inputs, outputs, vecs, vecs50, wordidx, embedding_size='50')
    
question_vectors_train, input_vectors_train, output_vectors_train, labels_train = question_vectors[:1700], input_vectors[:1700], output_vectors[:1700], labels[:1700]
question_vectors_test, input_vectors_test, output_vectors_test, labels_test = question_vectors[1700:], input_vectors[1700:], output_vectors[1700:], labels[1700:]


No vector present: not.a
No vector present: pair.you
No vector present: scratch.for
No vector present: made.she
No vector present: i|x
No vector present: state.for
No vector present: a|b
No vector present: ×
No vector present: b.amirali
No vector present: polynom
No vector present: ·
No vector present: ·
No vector present: b1xn
No vector present: addend
No vector present: «
No vector present: c*x^k
No vector present: »
No vector present: 5*x^8
No vector present: .please
No vector present: polynom
No vector present: «
No vector present: c*x^k
No vector present: »
No vector present: aryo
No vector present: aryo
No vector present: recently.he
No vector present: album.if
No vector present: latest.help
No vector present: //en.wikipedia.org/wiki/battleship_
No vector present: ×
No vector present: ×
No vector present: ×
No vector present: ×
No vector present: ×
No vector present: ×
No vector present: ×
No vector present: ×
No vector present: 'bend
No vector present: horizontaly
No vector pres

No vector present: '1
No vector present: '0
No vector present: black.polycarpus
No vector present: side-neighboring
No vector present: polycarpus
No vector present: borders.now
No vector present: polycarpus
No vector present: polycarpus
No vector present: intercal
No vector present: method.the
No vector present: i-th
No vector present: steps:1
No vector present: ascii-code
No vector present: i-th
No vector present: ascii-code
No vector present: i-th
No vector present: printed.you
No vector present: .you
No vector present: shakespeare.each
No vector present: operations.you
No vector present: usage.you
No vector present: variable=integer
No vector present: integer.let
No vector present: hq9+
No vector present: one-character
No vector present: accumulator.instructions
No vector present: ignored.you
No vector present: hq9+
No vector present: brainfuck
No vector present: brainfuck
No vector present: brainfuck
No vector present: →
No vector present: →
No vector present: →
No vector present: 

No vector present: ..
No vector present: «
No vector present: ..
No vector present: »
No vector present: ..
No vector present: one.the
No vector present: ..
No vector present: .initially
No vector present: 256-th
No vector present: n-gon
No vector present: n.the
No vector present: —
No vector present: zero.when
No vector present: —
No vector present: incoming.mr
No vector present: hold.mr
No vector present: well.what
No vector present: 1-st
No vector present: 86400-th
No vector present: polycarpus
No vector present: —
No vector present: —
No vector present: ≤
No vector present: message.you
No vector present: polycarpus
No vector present: xi.find
No vector present: p-th
No vector present: change.for
No vector present: polycarpus
No vector present: polycarpus
No vector present: polycarpus
No vector present: string.in
No vector present: ≤
No vector present: ≤
No vector present: ≤
No vector present: ≤
No vector present: |s|
No vector present: palindromes.a
No vector present: abacaba
No vec

No vector present: polycarpus
No vector present: polycarpus
No vector present: polycarpus
No vector present: ≤
No vector present: ≤
No vector present: polycarpus
No vector present: i-th
No vector present: .we
No vector present: i-th
No vector present: .the
No vector present: n-th
No vector present: y.every
No vector present: repeated.if
No vector present: students.write
No vector present: n-th
No vector present: .no
No vector present: palevny
No vector present: palevny
No vector present: each.the
No vector present: perelmanov
No vector present: perelmanov
No vector present: sheet.repeat
No vector present: perelmanov
No vector present: perelmanov
No vector present: bhtml
No vector present: bcss
No vector present: analogs.you
No vector present: bhtml
No vector present: tagname
No vector present: /tagname
No vector present: self-closing
No vector present: tagname/
No vector present: tagname
No vector present: /tagname
No vector present: tagnames
No vector present: tagnames
No vector prese

No vector present: boards.vasya
No vector present: vasya
No vector present: i-th
No vector present: 1-th
No vector present: i-th
No vector present: 1-th
No vector present: .now
No vector present: vasya
No vector present: l.two
No vector present: 1000000007
No vector present: codeforces
No vector present: codeforces
No vector present: size.you
No vector present: j.since
No vector present: king.we
No vector present: k2l
No vector present: k2l
No vector present: k2r
No vector present: letters.dwarf
No vector present: watch.each
No vector present: break.after
No vector present: vary.before
No vector present: versa.dwarf
No vector present: totalizer
No vector present: results.the
No vector present: lot.one
No vector present: permutation.happy
No vector present: features.algorace
No vector present: cars.the
No vector present: i-th
No vector present: destination.pmp
No vector present: ’
No vector present: experiences.the
No vector present: i-th
No vector present: s-th
No vector present: exper

...

No vector present: conditions.otherwise
No vector present: space-separated
No vector present: ≤
No vector present: —
No vector present: 1000000007
No vector present: —
No vector present: problem.please
No vector present: specificator
No vector present: с++
No vector present: cout
No vector present: i64d
No vector present: specificator
No vector present: —
No vector present: 1000000007
No vector present: —
No vector present: —
No vector present: —
No vector present: i-th
No vector present: i-th
No vector present: rod.if
No vector present: —
No vector present: —
No vector present: dravde
No vector present: regarder
No vector present: dravde
No vector present: dravde
No vector present: dravde
No vector present: —
No vector present: polycarpus
No vector present: command.please
No vector present: n·k
No vector present: ≤
No vector present: ≤
No vector present: k-th
No vector present: k-th
No vector present: operation.as
No vector present: space-separated
No vector present: —
No vector prese

No vector present: —
No vector present: them.the
No vector present: characters.if
No vector present: space-separated
No vector present: dominoes.if
No vector present: space-separated
No vector present: —
No vector present: non-negative.if
No vector present: 1000000007
No vector present: —
No vector present: φ
No vector present: φ
No vector present: φ
No vector present: φ
No vector present: times.each
No vector present: space-separated
No vector present: ≥
No vector present: —
No vector present: representaion
No vector present: —
No vector present: ainta
No vector present: obtain.in
No vector present: k-th
No vector present: k-th
No vector present: k-th
No vector present: k-th
No vector present: any.please
No vector present: с++
No vector present: cout
No vector present: i64d
No vector present: count_online_friends
No vector present: 1000000007
No vector present: —
No vector present: —
No vector present: space-separated
No vector present: —
No vector present: i-th
No vector present: j-t

No vector present: with.if
No vector present: —
No vector present: —
No vector present: —
No vector present: t.in
No vector present: —
No vector present: —
No vector present: aboce
No vector present: line.in
No vector present: space-separated
No vector present: i-th
No vector present: i-th
No vector present: them.if
No vector present: —
No vector present: —
No vector present: —
No vector present: .remember
No vector present: —
No vector present: —
No vector present: j-th
No vector present: ≤
No vector present: ≤
No vector present: —
No vector present: j-th
No vector present: —
No vector present: answers.if
No vector present: —
No vector present: once.in
No vector present: order.each
No vector present: graph.next
No vector present: ≤
No vector present: ≤
No vector present: ≤
No vector present: ≤
No vector present: .edges
No vector present: kitayuta
No vector present: 10007
No vector present: learn.in
No vector present: space-separated
No vector present: order.if
No vector present: ×
No 

No vector present: «
No vector present: »
No vector present: .if
No vector present: j-th
No vector present: —
No vector present: —
No vector present: —
No vector present: .if
No vector present: —
No vector present: i-th
No vector present: maximum-weight
No vector present: 1.note
No vector present: hypothetical—your
No vector present: i-th
No vector present: —
No vector present: —
No vector present: n-th
No vector present: —
No vector present: them.if
No vector present: n-th
No vector present: —
No vector present: —
No vector present: —
No vector present: —
No vector present: books.it
No vector present: 2·109
No vector present: -1.otherwise
No vector present: —
No vector present: —
No vector present: n.in
No vector present: i-th
No vector present: i-th
No vector present: edge.the
No vector present: i-th
No vector present: —
No vector present: i-th
No vector present: i-th
No vector present: —
No vector present: —
No vector present: —
No vector present: —
No vector present: limak
No vecto

No vector present: —
No vector present: one.if
No vector present: false-rumoured
No vector present: space-separated
No vector present: synchrophasotron
No vector present: synchrophasotron
No vector present: synchrophasotron
No vector present: .the
No vector present: synchrophasotron
No vector present: desintegrated
No vector present: —
No vector present: p'i
No vector present: ≤
No vector present: p'i
No vector present: ≤
No vector present: —
No vector present: vasya
No vector present: vasya
No vector present: i-th
No vector present: —
No vector present: .otherwise
No vector present: —
No vector present: —
No vector present: i-th
No vector present: —
No vector present: —
No vector present: balls.in
No vector present: —
No vector present: —
No vector present: berlbury
No vector present: milllimeters.in
No vector present: —
No vector present: berlbury
No vector present: —
No vector present: roads.in
No vector present: ≤
No vector present: ≤
No vector present: —
No vector present: solutio

In [73]:
print(type(labels))

<class 'numpy.ndarray'>


## Config

In [74]:
max_question_length = 300
max_input_length = 50
max_output_length = 50
embedding_vector_length = 32
n_epochs = 5

## Training

In [75]:
x = np.array([1,2,3,4,4,5,6], dtype=int)
y = np.array([4,5,6], dtype=int)
z = np.concatenate([x,y]).astype(float)
z

array([1., 2., 3., 4., 4., 5., 6., 4., 5., 6.])

In [110]:
def build_model(question_vectors_train, input_vectors_train, output_vectors_train, labels_train, vocab_size, max_question_length, max_input_length, max_output_length, embedding_vector_length, n_epochs):
    
    question_vectors_sequence = sequence.pad_sequences(question_vectors_train, maxlen=max_question_length)
    input_vectors_sequence = sequence.pad_sequences(input_vectors_train, maxlen=max_input_length)
    output_vectors_sequence = sequence.pad_sequences(output_vectors_train, maxlen=max_output_length)
    
#     data_1 = shape (None, 1150,50) 
#     input_1 = Input(shape=data_1.shape)
    
    model = Sequential()

    model.add(LSTM(50, input_shape=(None, None, 50)))

    model.add(Dense(3, activation = "softmax"))

    model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

#     print('shapes', question_vectors_sequence.shape, input_vectors_sequence.shape, output_vectors_sequence.shape)
#     print('q:', question_vectors_sequence[0])
#     print('i:', input_vectors_sequence[0])
#     print('o:', output_vectors_sequence[0])
#     print('conc:', np.concatenate([question_vectors_sequence[0:2], input_vectors_sequence[0:2], output_vectors_sequence[0:2]], axis=1))
    X = np.concatenate([question_vectors_sequence, input_vectors_sequence, output_vectors_sequence], axis=1)
    model.fit(X, labels_train, epochs=n_epochs, batch_size=64, verbose=1)

    print('SUMMARY:', model.summary())
    return model

In [111]:
model = build_model(question_vectors_train, input_vectors_train, output_vectors_train, labels_train, vocab_size, max_question_length, max_input_length, max_output_length, embedding_vector_length, n_epochs)

ValueError: Input 0 is incompatible with layer lstm_12: expected ndim=3, found ndim=4

## Testing

In [20]:
def test_model(model, question_vectors_test, input_vectors_test, output_vectors_test, labels_test, max_question_length, max_input_length, max_output_length):
    question_vectors_sequence = sequence.pad_sequences(question_vectors_test, maxlen=max_question_length)
    input_vectors_sequence = sequence.pad_sequences(input_vectors_test, maxlen=max_input_length)
    output_vectors_sequence = sequence.pad_sequences(output_vectors_test, maxlen=max_output_length)

    X = np.concatenate([question_vectors_sequence, input_vectors_sequence, output_vectors_sequence], axis=1)
    scores = model.evaluate(X, labels_test)
    print("Accuracy: %.2f%%" % (scores[1]*100))

In [21]:
test_model(model, question_vectors_test, input_vectors_test, output_vectors_test, labels_test, max_question_length, max_input_length, max_output_length)

457/457 [==============================] - 1s 3ms/step
Accuracy: 43.54%
